# X-ray Data Analysis of point sources

<!--<div class="alert alert-block alert-info">
</div>-->

Today we will perform some basic analysis on an X-ray dataset of GW170817. The goal is to understand how the raw data from a telescope must be processed to extract flux densities, which can then be used for modeling. This observation was taken with the Chandra X-ray Observatory on 2017 December 3. The data were independently published by several teams, including Margutti et al. (2018) and Ruan et al. (2018). As we will see in the extension activity, X-ray observations of GW170817 and other multi-messenger sources probe the fastest-moving ejecta from the merger. In the case of GW170817, these X-ray data helped prove that GW170817 launched a short GRB-like relativistic jet, viewed off-axis. For additional background, see Margutti & Chornock (2021) for an overview of the wide variety of science enabled by electromagnetic observations of GW170817.

__Goals:__ Understand the very basics of data formats in X-ray Astronomy; get familiar with event files; understand the difference between an event and a photon; learn to filter events in time, space and energy; source detection; source counts and count-rates. 

## 1) Getting started

In this tutorial, we will use CIAO (Chandra's custom data reduction software) and ds9. ds9 is a very useful tool for interacting with astronomical images (particularly those in FITS format), and we recommend that you download it for your personal use if you expect to do data analysis in the future. If ds9 is not compatible with your operating system, you can access an online version here: https://js9.si.edu/. For this tutorial, we will use js9 integrated with this notebook. Note that some of the js9 menu options are different from the standalone application; please ask the TA for help if anything is unclear.

All public Chandra data can be accessed online via the data archive browser: http://cda.harvard.edu/chaser/. Each Chandra observation is associated to an ID; you can search for data by this ID number, or by the target name, coordinates, observation date, etc. In this case, we are using the observation with ID= 20860. To expedite things, we have already downloaded the dataset that we will use in this tutorial.

The data for this activity is contained in the file package_7697_220112160139.tar. In the directory where you would like to work, run:

In [ ]:
%%bash
tar -xvf package_7697_220112160139.tar

You should now see a new directory with the observation ID for the downloaded dataset.

## 2) Structure of Chandra ACIS-S data
We will work with Chandra ACIS-S data. Chandra has different instruments onboard. We will use ACIS-S that has the best performance when it comes to detect and characterize faint sources (we will see that the gravitational wave source is one of them).

Go inside the /20860/ directory and list the items there.

In [ ]:
%%bash
cd 20860
ls

Go into the /primary/ folder and list the content: it contains the results from a dataset that has been already re-processed. The automatic reprocessing is good for the majority of cases (however for publication level products it is always good practice to re-process the data). For your /20860/ data the automatic processing is good enough and we will not reprocess the data ourselves. Most of the files that we need are there already.

In [ ]:
%%bash
ls 20860/primary

Most of the files in the directory are fits files (that we will use for our analysis) but we also have .jpg files that offer a very quick view of the results of the observation -but they are not useful for almost anything else-

Now go into the /secondary/ folder and list:

In [ ]:
%%bash
ls 20860/secondary

This folder contains a rich set of information about the specifics of the instrument at the time of data acquisition which are necessary to process the data correctly. You can think about /primary/ and /secondary/ as already re-processed and cleaned files (primary) vs. original data products and instrumental info (secondary).

In [ ]:
# ALTERNATIVELY: open ds9 in on your laptop or in the VNC session, load the files there (please refer to instructions in https://renkulab.io/gitlab/saasfee2022/course-materials/) load the files there).
# NOTE: if you are using ds9, or downloading regions from js9 to your latop, do not forget to upload them back to the jupyterlab notebook (see an icon on top left)

import jpjs9
J = jpjs9.JS9(id='main')
J.display()

Commands below will open in js9 the two files:  /primary/acisf20860_000N002_evt2.fits.gz and /secondary/acisf20860_000N002_evt1.fits.gz. 

In [ ]:
J.Load('20860/primary/acisf20860N002_evt2.fits.gz')

In [ ]:
J.Load('20860/secondary/acisf20860_000N002_evt1.fits.gz')

Align the two frames by going to View --> Sync Images and check the boxes for alignment, zoom, scale. (If you are using the desktop version of ds9: go into the menu Frame --> Match --> Frame --> WCS.) Blink the two frames. Remember to use log scale in both frames and adjust the contrast. What do you notice? This simple exercise is meant to make you appreciate the “beauty” of data re-processing and cleaning. Show the result to your instructor.

In [ ]:
# OPTIONAL: if you are using renku and the network is a bit slow, the previous command might take long and time-out.
# In this case, try to bin the image before loading in js9

#!dmcopy "20860/secondary/acisf20860_000N002_evt1.fits.gz[bin x=1:8192:4,y=1:8192:4]" 20860/secondary/acisf20860_000N002_evt1_binned.fits clobber=yes

# using "parentFile" option will ensure that js9 refers operations to the original file
#J.Load("20860/secondary/acisf20860_000N002_evt1_binned.fits", {'parentFile': "20860/secondary/acisf20860_000N002_evt1.fits.gz"});

Next, we are going to use astropy to further explore the structure and contents of this fits file:

In [ ]:
from astropy.io import fits
evt2 = fits.open('20860/primary/acisf20860N002_evt2.fits.gz')
evt2.info()

js9/ds9 visualizes the image, astropy.io.fits shows the “bones” of the structure of the fits file. But it is the very same file viewed in two different ways.

From the header of the first extension, named EVENTS, find out what is the exposure time of this observation. What do you find? A few lines below, you can see that there is a “dead-time” correction. What is that?

In [ ]:
evt2[1].header # display full header for the first extension (in this case 'EVENTS')
#evt2['EVENTS'].header['EXPOSURE'] # you can also display a specific value stored in the header (in this case the exposure length in seconds)

Now we will look at the data contained in the EVENTS extension:

In [ ]:
evt2[1].data

Each line in the table represents an “event”. This list of events has been already “cleaned” -the comparison of the evt1 to the evt2 file should have clarified what this means- and you can think about each line in the table as a count associated to some photon arriving at the detector.

IMPORTANT: in X-ray Astronomy we use photon counting detectors, so we can really count the number of photons that arrived to the detector. For each photon (or things thought to be a photon), we record different info, including: the time of arrival, the location in the detector where the event arrived and the “grade”. In the table above you will find a column that contains the grade information for each event that passed the re-processing cut, and it it thus very likely associated with photons arriving at the detector.

An __event__ = anything that creates a displacement of charge in the CCD (i.e. in your detector).

The __grade__ is a number assigned to every event based on which pixels in its 3x3 island are above their threshold value. The initial grade is assigned by on-board processing, which first finds local maxima, then analyzes the values of the surrounding 3x3 neighboring pixels. Based on this pattern, the event is
assigned a grade. For example, a single-pixel event has a grade of 0. There are grades that are typical of photons (Good!) and grades which are created when something else happens (bad!), like particles etc. These events will be discarded in the processing (i.e. will not pass the quality cut during the processing).

<img src="fig1.png" class="bg-primary" width="800px">

__Figure 1:__ Chandra flight grades. Do not pay attention to the color (which just describes the mapping of Chandra grades into ASCA grades, not important for you).

## 4) More on evt files

Go back to the js9 view of the evt2 file and zoom out (or bin out if you are using ds9). This way you will be able to appreciate the structure of the CCDs that we used. You should see something like this:

<img src="fig2.png" class="bg-primary" width="300px">

__Figure 2:__ binned-out view of the evt2 file.

<img src="fig3.png" class="bg-primary" width="800px">

__Figure 3:__ ACIS focal plane.

Figure 3 above will help you understanding what you are looking at. We are using S2 and S3 on ACIS-S and I2 and I3 from ACIS-I. From this simple exercise you can clearly appreciate the different levels of background of the different chips.

## 5) Filtering event files in energy and in space.

Using ds9 we can filter the events into a particular energy range using “Bin” (for js9, the binning functionality is under the View menu). However, instead we will do something a little more refined. The goal is to create a new evt file that is filtered in energy and in a given region.

We will use the CIAO task dmcopy. To find out more about the syntax, you can use the online help:
http://cxc.harvard.edu/ciao/ahelp/dmcopy.html

In [ ]:
%%bash
cd 20860/primary
dmcopy "acisf20860N002_evt2.fits.gz[ccd_id=7][energy=500:8000][bin x=3848:4360:1.0,y=3860:4372:1.0]" evt_500_8000_img.fits

Comments:

• [ccd_id=7] → we are telling CIAO that the x an y coordinates are on the CCD with id=7. Go back to Fig. 3 and see if it makes sense. Our source of interest is in S3.

• [energy=500:8000] this means: filter in energy between 0.5 keV and 8 keV, which is the range where Chandra is better calibrated. 

• [bin x=3848:4360:1.0,y=3860:4372:1.0] this means: filter out between those xmin and xmax, ymin and ymax. In this way we are effectively reducing the size of the evt file that we will use for the following analysis, so that any operation that follows will be appreciably faster. However, we need to make sure that the the size of the final evt file is not too small around the source, or we will not have enough statistics to understand the properties of the background around the source, and hence do a reliable source detection.

• evt_500_8000_img.fits: this is the name of the output file. It is good practice to save it with a name that reflects the nature of its content.
Open evt_500_8000_img.fits with js9 and visualize the content (always log scale with X-rays!). Check the size of the new file and that the file has been filtered in energy (compare the same region of the original evt file in js9).

Check the structure of the new evt file with astropy.io.fits. How many extensions do you have? Do you notice differences with respect to the original evt file? This is because this fits file is an image (this is why I put an .img extension), i.e. we have lost the “table” structure, which means that we lost the info about the details of the arrival time of each photon. We lost the “time” coordinate and merged everything into an image.

In [ ]:
evt_img = fits.open('20860/primary/evt_500_8000_img.fits')
evt_img.info()

As a final step we will need to compute the PSF (Point Spread Function) at each position in our image. We will use the task: mkpsfmap. As always you can find out more about the task going online on the CIAO help page.

In [ ]:
%%bash
cd 20860/primary
mkpsfmap evt_500_8000_img.fits outfile=evt_500_8000_psfmap.fits energy=1.4967 ecf=0.90 clobber=yes

Comments:

• mkpsfmap expects as input a fits file, in this case evt_500_8000_img.fits.

• The output file is also a fits file. I gave it the name evt_500_8000_psfmap.fits, so that it is easy to
remember which image is the PSF map for.

• Energy=1.4967 (keV) specifies the energy at which to compute the PSF. This is important because the PSF varies as a function of energy, so we need to specify a “typical” energy for computing the PSF. The best choice will depend on the nature of the source we are interested in. From experience I know that an energy ~1.5 keV is fine for the PSF map for the majority of cases.

• Ecf= encircled energy fraction. Here we are specifying that we want the output number to be the radius, (in arcsec!) that contains the 90% of energy.

Now open the newly created fits file evt_500_8000_psfmap.fits with js9. You should see something like in Figure 4 below:

<img src="fig4.png" class="bg-primary" width="300px">

__Figure 4:__ PSF map

In [ ]:
# here we make a new view of JS9, for viewing the PSF
J_psf = jpjs9.JS9('psf')
J_psf.display()

In [ ]:
J_psf.Load("20860/primary/evt_500_8000_psfmap.fits")

If you go with your mouse over the map you will see that the values vary from ~0.9'' in the central black region to ~1.7'' in the outer regions. What you are looking at is the effect of deterioration of the PSF as we go further away from the on-axis condition . At the center an angular region as small as 0.9'' contains 90% of energy, while going further away the 90% ECF becomes significantly larger.

Note: this angular resolution is AMAZING for X-ray astronomy. No other satellite can do better. Other satellites like Swift-XRT or XMM have much larger PSFs, of several arcsec (XRT has an 18 arcsec PSF)!

## Counts and count-rates from a region (for our source, and the background).
We will create two regions, one for our source of interest and one that we will use to estimate the background (i.e. the number of spurious counts).
Our source of interest is GW170817, which, from optical observations we know has coordinates: 

RA= 13:09:48.08, 
dec= -23:22:53.3

In js9, create a region of 3 arcsec around GW170817 and name it GW170817_3arcsec_optical.reg. (Go to Regions --> Circle, double click on the circle that appears in the center of the image, enter the desired coordinates and radius, and click Apply. Alternately, in the desktop ds9, go to Edit --> Region, click on the image so a green circle appears, double click on the circle, enter the desired coordinates and radius, and click Apply.) Save a version in fk5 coordinates (GW170817_3arcsec_optical.reg) and a version in physical coordinates (GW170817_3arcsec_optical_physical.reg) by editing the file name and clicking Save at the bottom of the same Edit region dialog box. When saving, under Options make sure that Save comments is checked and Save JSON object and Save dcoord regions are unchecked. (In ds9: go to Regions --> Save.) Note that this saves the .reg file on your local machine; you can easily upload this into your renku session by clicking on the upload button. 

__IMPORTANT:__ When you have uploaded the .reg files, double click on GW170817_3arcsec_optical_physical.reg to open it in your renku session. The first line of the .reg file specifies the Region file format as JS9 version 1.0. You must manually change this to Region file format: DS9 version 4.1 and save it before proceeding with the dmlist step below, otherwise CIAO will not know how to interpret the file correctly. 

Now create a circular background region. Important specs for the background region:

• Must be far away from any source that you can identify with your eyes in the js9 image when
the scale is in log units.

• Must be far away from chip gaps.

• Must be reasonably close to the source of interest, and preferentially in the centre of the chip.

• Must be significantly larger than the source region, to protect ourselves against statistical
fluctuations. Select a radius of ~35 arcsec or more. Do you understand what this means?

As before save a version of the background region in fk5 coordinates (bk.reg) and one in physical coordinates (bk_physical.reg). Remember to also manually edit the .reg file to set the Region file format to DS9 version 4.1. You can also load and save region files from within the notebook, e.g.:

In [ ]:
# OPTIONAL: you can use JS9 save region functionality to avoid passing file through the local machine. 
# If you have extra time, try to make it work (beware you'll have to select and save only necessary regions)
#J.SaveRegionFile("GW170817_3arcsec_optical.reg")

In [ ]:
%%bash
cd 20860/primary
cp -fv ../../*.reg . # This line may not be needed if you already uploaded or saved your file directly to /20860/primary 
dmlist 'acisf20860N002_evt2.fits.gz[energy=500:8000][sky=region(GW170817_3arcsec_optical_physical.reg)]' counts

The CIAO dmlist task, with the option “counts” at the end, will produce as output the number of counts that are in the evt file that you provide as input, after applying all the filters you want. Each filter is defined in a “[filter]” structure. Here we are filtering in energy with [energy=500:8000](which energy band is this??) and we will only accept the counts within the SN2014C_3arcsec_optical_physical.reg region. Another way to visualize the number of photons in a region is by double clicking on the region file, Analysis, Statistics.

Now estimate your “background level” in the source region, by calculating the number of 0.5-8 keV counts that you have in the background region, and then renormalize them by the size of the source region. The goal is to be able to answer to the question: how many 0.5-8 keV counts from the background do you expect to have in the source region? Compare it to the number of photons that you effectively have in the source region.

In [ ]:
%%bash
cd 20860/primary
dmlist 'acisf20860N002_evt2.fits.gz[energy=500:8000][sky=region(bk_physical.reg)]' counts

In [ ]:
# sample renormalization to get expected number of background counts
expected_counts = 655*(3/35)**2
print(expected_counts) 

Note that this is much less than the number of counts in our actual source region -- so we have a significant detection! We can also convert to an average count rate by subtracting the number of expected background counts from the number of observed counts in our source region, and then dividing by the exposure length (do you remember how to extract this from the image header, as we did above?)

In [ ]:
from astropy.io import fits
evt2 = fits.open('20860/primary/acisf20860N002_evt2.fits.gz')
print((114-expected_counts)/evt2['EVENTS'].header['EXPOSURE']) # units are counts per second

If you have chosen a good background region, this should be consistent with the count rate reported in Margutti et al (2018). In the next section, we'll explore the image a bit more and provide some tips for improving your background region if necessary.

## 7) Source Detection with wavedetect

Goal: we want to find all the point-like sources in our image. Since our input is an image filtered in the energy range 0.5-8 keV, the output will be a list of point-like sources in the 0.5-8 keV range. We will use the wavedetect task within CIAO.

In [ ]:
%%bash
cd 20860/primary
wavdetect infile="evt_500_8000_img.fits" outfile="evt_500_8000_src.fits" scellfile="evt_500_8000_scell.fits" imagefile="evt_500_8000_imgfile.fits" defnbkgfile="evt_500_8000_nbgd.fits" regfile="evt_500_8000_src.reg" scales=" 1.0 2.0 4.0 8.0 16.0" clobber=y sigthresh="4e-06" psffile="evt_500_8000_psfmap.fits"

There are many different algorithms that can be used to detect sources. Here we will use wavelets. For fainter sources, it is possible to implement very basic and simple Poisson statistics to get solid results in the realm of source detection in the regime of very low number statistics.

IMPORTANT: A source detection is always a statement about the background/noise. In a nutshell, we say that we have detected a source IF the observed counts at a certain location in the sky are in excess with respect to what we expect from the background alone (the background can be astronomical or instrumental, it does not matter here). (We will learn more in class in the next 2 lectures). Wavelets allows you to explore such deviations with respect to the background over different pixel scales.

Comments about the syntax:

• infile: this is your filtered image

• outfile: it is going to be a fits file that contains the list of sources found. I call it “_src”

• regfile: region files with the regions associated to the detected sources. The sizes of these
regions are optimized by the algorithm to maximize the signal-to-noise in the region.

• Scales: all different pixels scales over which I want to run the tool and see if I find a source. Is it
clear why using different scales I might be able to find sources that were not appearing obvious
before?

• clobber=yes. This is just a parameter that allow the tool to overwrite on files.

• Sighthresh: this is the signal threshold to define something a detection. The number I chose is good enough not to throw away potential detections, but also ok in the sense of not having too many false positives.

• Psffile: this is the psfmap that we just generated with mkpsfmap. This is a necessary input as this is the way we are telling the algorithm how a point source looks like on the detector.

Open evt_500_8000_img.fits with js9 (delete the psfmap frame), log scale. Now we want to see which sources have been detected: “Region” “Load”, select: evt_500_8000_src.reg
Now open evt_500_8000_src.reg with your favorite text editor to see the structure of the file (it is a list of regions).

Is GW170817 detected in this Chandra observation? Load your GW170817_3arcsec_optical.reg region and see if you have a green region around it, which means: detected! (The answer is obvious for GW170817, but it is not always this obvious, especially if we push the instrument to the very limit of its performance, which is necessary to track the full light curve evolution of gravitational wave sources.)

The file evt_500_8000_src.fits contains all the relevant information for each source. Sources appear in the file in order of detection significance, from the most significant to the least:

In [ ]:
from astropy.io import fits
src_img = fits.open('20860/primary/evt_500_8000_src.fits')
src_img.info()

You have two extensions. Look at the data in the SRCLIST extension. Each line of this table is a detected source. 

In [ ]:
src_img[1].data

The table contains important information, including:

• Coordinates of the sources detected (RA, dec) and their absolute uncertainties (RA_err, dec_err)

• X,Y: coordinates on the detector and error bar.

• NET_COUNTS: background subtracted counts and uncertainty.==> Net-counts: total counts in
the region – background counts in the region. Since we filtered the events into the 0.5-8 keV
energy range, these are the source counts in that energy band.

• BKG_COUNTS: counts from the background in the region and error bar.

• RATE columns: discard. This is not a count-rate.

• EXP-TIME: 1. Discard.

• SRC_SIGNIFICANCE: this is an important column. It gives you the statistical information
about the significance of the detection in units of Gaussian sigma. We will learn more about the meaning of Gaussian sigmas in the next lectures. For now, just learn that something with >= 3 sigma detection means: pretty solid detection.

• PSF-SIZE (in pixels!): at the location of the source. It tells you the angular resolution of your detector at that location on the detector.

• You also have some info on the shape of the regions. In js9 you can load the evt_500_8000_img.fits file together with the region file evt_500_8000_src.reg. If you click o one of the regions, you will find information about the size and shape of the region. Note that not all the regions are circular.



Load the GW170817_3arcsec_optical.reg into js9 (use a different color for the GW170817 region – you
can change the color of the region double clicking on the region and changing the settings in the menu that appears).

Final step: identify GW170817 in the list of sources in evt_500_8000_img.fits. Which source number is GW170817? What is the easiest way to identify GW170817 in the list?

Finally load your background region on the js9 and check if it contains any of the detected sources. If it does, it means that your initial choice of background is not an ideal one and you will need to redefine the background region and re-do your estimates above with a new background region that does NOT contain any source (and it is far away from the chip gaps). The background region needs to be source- free!!

Note on “event” vs. “counts” vs. “photons”: there is a reason why until now we have been speaking of counts and not of “photons” yet. A quick recap:

Event= anything that creates a displacement of charge in the CCD. By assigning a grade, we effectively filter out events that have high probability of not being photons.

Count= an event that is produced by photon(s), but not necessarily one photon.

What happens in CCDs that are photon counting detectors, two photons that arrive very close in time to to the same pixel (or close too), can be mistakenly interpreted as one single photon with higher energy. This effect is called pile-up. Pile-up:

• Is important only when the rate of photons arriving to the detector is large (how large depends on the CCD and on the mode of operation of a CCD). For the sources that we will use and our set up, pile up is not important.

• Pile-up affects different portions of the PSF of a source in a different way: since most of the photons will tend to concentrate in the core of the PSF, and fewer photons will be part of the PSF tail, pile-up more severely affects the core of the PSF [this actually gives us an easy way for correct for pile-up, see reading in the folder].

• By recording 2 photons as 1 photon, pile-up leads to an underestimation of the actual photon rate by the source. This directly affects the light-curve of an object.

• By recording 2 lower energy photons as 1 higher energy photon, pile-up also causes a deformation of the spectrum (a spectral hardening in this case, as it goes in the direction of more energy).

__VERY IMPORTANT: in X-ray astronomy you always have to report the energy band you use to extract your results. Reporting a number of counts or count-rate or any other quantity without providing the energy range is effectively is of no scientific use!!!!__

The final step in the data reduction process is to convert the number of counts we received to a flux density. To do this properly, X-ray astronomers need to use information about the __spectrum__ of the X-ray emission within the given energy range. (This can either be measured directly from the data if the source has enough counts, or a specific model must be assumed, e.g. a power law.) This is beyond the scope of today's activity, although we provide instructions on how to install and use a tool called Xspec to do this on your own if you are interested. The outcome of the spectrum analysis allows us to accurately convert from the observed count rate to a flux density, which can then be used for science. For the extension activity, we will explore some of the science that can be done with the full X-ray light curve of GW170817 (in combination with radio and late-time optical data).